In [ ]:
#Reference Doc: #https://jovian.com/altamashwaseem04/scraping-top-selling-games-on-steam#C2

In [ ]:
# Create a new python environment 
# Python 3.8.2

In [ ]:
#%apt update
#%apt install chromium-chromedriver --quiet
#%pip install selenium --quiet
#%pip install pandas --upgrade --quiet

In [1]:
!pip install jovian --upgrade --quiet

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import pandas as pd
import re
import time

In [2]:
def create_driver(url):
  '''Takes the url as an input, creates the webdriver and returns the driver with the page'''
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome(options=options)
  wd.get(url)
  return wd

In [3]:
url='https://store.steampowered.com/charts/topsellers/US/2023-7-18'
wd=create_driver(url)

# OTHER URLS TO SCRAPE: 
# IF POSSIBLE SCRAPE THE TOTAL VISITORS/TOTAL INCOME/TOTAL GAMES SOLD from steam.com ⭐
# Total & Active Player Counts: https://store.steampowered.com/charts/ 
# Most Played: https://store.steampowered.com/charts/mostplayed

In [4]:
wd.get(url) # open the url in browser # collect response

In [5]:
wd.title # get the title of the page # Checks if data is loaded

'Weekly Top Sellers - For Tue, July 18, 2023 - Tue, July 25, 2023'

In [6]:
import time

time.sleep(3) #below line fails without this idk

# Find the button element using one of the available locating strategies

button = wd.find_element(By.CLASS_NAME, 'DialogButton._DialogLayout.Primary.Focusable')  # Replace 'button_id' with the actual ID of the button

# Click the button

button.click() # helps get 100 results instead of 20

In [7]:
wd.get(url)

In [8]:
wd.title

'Steam Charts'

In [9]:
def scroll_page(wd):
  '''Takes the driver as an input and simulates the scrolling to get all the games'''  
  SCROLL_PAUSE_TIME = 2

  # Get scroll height
  last_height = wd.execute_script("return document.body.scrollHeight")

  for i in range(6):
    # Scroll down to bottom
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
     # Calculate new scroll height and compare with last scroll height
    #If the new scroll height is equal to the previous scroll height, it means the page has reached the end, and the loop is terminated.
    #Otherwise, the new scroll height becomes the last scroll height, and the loop continues
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
  return

In [10]:
scroll_page(wd)

In [11]:
'''games_rows = wd.find_element(By.ID, 'search_resultsRows')
games = games_rows.find_elements(By.TAG_NAME,'a')
type(games)'''

"games_rows = wd.find_element(By.ID, 'search_resultsRows')\ngames = games_rows.find_elements(By.TAG_NAME,'a')\ntype(games)"

In [12]:
def get_games(wd):
  '''Takes the driver as an input gives and returns a selenium web element with all the games'''
  game_rows = wd.find_elements(By.CLASS_NAME, 'weeklytopsellers_TableRow_2-RN6')
  #games_rows = wd.find_element(By.ID, 'search_resultsRows')
  #games = game_rows.find_elements(By.TAG_NAME,'tr')
  return game_rows

In [13]:
game_rows=get_games(wd)

In [14]:
type(game_rows)

list

In [15]:
games = []

for game in game_rows:

    games.append(str(game.text).split('\n'))

In [16]:
games

[['1', 'Counter-Strike: Global Offensive', 'Free To Play', '0 572'],
 ['2', 'Remnant II', '$49.99', '▲ 9 3'],
 ['3', "Baldur's Gate 3", '$59.99', '▲ 1 32'],
 ['4', 'Steam Deck', '$399.00', '▼ 2 74'],
 ['5', 'BattleBit Remastered', '$14.99', '▼ 2 6'],
 ['6', 'Apex Legends™', 'Free To Play', '▲ 3 142'],
 ['7',
  'Call of Duty®: Modern Warfare® II',
  '-45%',
  '$69.99',
  '$38.49',
  '0 53'],
 ['8', 'Destiny 2', 'Free To Play', '▲ 5 211'],
 ['9', 'Team Fortress 2', 'Free To Play', '▼ 4 34'],
 ['10', 'The Sims™ 4', 'Free To Play', '▲ 20 162'],
 ['11',
  "Tom Clancy's Rainbow Six® Siege",
  '-60%',
  '$19.99',
  '$7.99',
  '▲ 6 401'],
 ['12', 'War Thunder', 'Free To Play', '▲ 6 317'],
 ['13', 'Red Dead Redemption 2', '-67%', '$59.99', '$19.79', '▲ 10 190'],
 ['14', 'Grand Theft Auto V', '-63%', '$39.98', '$14.99', '▲ 1 445'],
 ['15', 'Black Desert', '$9.99', '▲ 1 126'],
 ['16', 'Rust', '$39.99', '▼ 2 422'],
 ['17', 'Lost Ark', 'Free To Play', '▼ 9 81'],
 ['18', 'The Elder Scrolls® Online',

In [18]:
weekly_top_sellers = pd.DataFrame(games, columns=['col1','col2','col3','col4','col5','col6'])

In [19]:
weekly_top_sellers=weekly_top_sellers.reset_index(drop=True)

In [20]:
weekly_top_sellers.to_csv("Weekly_Top_Sellers.csv",index=False)